In [84]:
#Grace
import sys
sys.path.append('src/')
import preprocessing as pp
from importlib import reload # reload 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV

#reload(pp)
df = pd.read_csv("superheroes_nlp_dataset.csv")

### NLP PRE-PROCESSING
# replace nulls
df = pp.fill_na_columns(df)

# lowercase
df = pp.lowercase_columns(df)

# remove stopwords
df = pp.remove_stopwords(df, ['history_text', 'powers_text'])

# lematize
df = pp.lemmatize_columns(df, ['history_text', 'powers_text'])
pd.set_option('display.max_colwidth', None) #Allows you to see the whole row


In [85]:
# Removing Empty Lists
#df[df.history_text.apply(lambda c: c != [])]
#History
df = df[df.history_text.apply(lambda c: c != [])] #90
#Powers 
#df = data[data.powers_text.apply(lambda c: c != [])] #Too much missing 375 

#Limit Data to Creators who have count > 10
#8 Different Comic Creators
creators_list = ['marvel comics', 'dc comics', 'shueisha','dark horse comics', 'lego', 'george lucas','image comics','nbc - heroes']
df = df[df.creator.isin(creators_list)]

In [86]:
df.creator.value_counts()

marvel comics        583
dc comics            414
shueisha              35
dark horse comics     26
lego                  22
george lucas          18
image comics          16
nbc - heroes          15
Name: creator, dtype: int64

In [87]:
#Using just history_text
X = data['history_text'].astype('string')
y = data['creator']#.to_numpy()

# create our tfidf vectorizer stuff
vectorizer = TfidfVectorizer(min_df = 10, ngram_range=(1, 2), max_features=10000 ,stop_words = {'one', 'later', 'would', 'also'})
history_tfidf = vectorizer.fit_transform(X)
history_tfidf =  history_tfidf.toarray()

In [88]:
#  create a dictionary mapping the tokens to their tfidf values
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
tfidf = pd.DataFrame(columns=['tfidf']).from_dict(
                    dict(tfidf), orient='index')
tfidf.columns = ['tfidf']
tfidf.sort_values(by=['tfidf'], ascending=True).head(10)

,tfidf
time,1.749577
new,1.757096
however,1.841709
power,1.845824
him,1.858272
became,1.925180
eventually,1.943198
life,1.954627
man,1.977884
member,2.031036


In [89]:
#y = df['creator']#.to_numpy()
#X = df.loc[:, ['history_text', 'powers_text']]
# one hot encode our labels and features for prediction sake
le = LabelEncoder()
oe = OneHotEncoder(sparse=False)

# integer_X = le.fit_transform(vectorizer.get_feature_names())
# integer_X = integer_X.reshape(len(integer_X), 1)
# onehot_X = oe.fit_transform(integer_X)

integer_y = le.fit_transform(y)
integer_y = integer_y.reshape(len(integer_y), 1)
onehot_y = oe.fit_transform(integer_y)


# get back the original labels
# inverted = le.inverse_transform([np.argmax(onehot_y[0])])
# this is how we'd get out the actual label names of the encoder
# for i in onehot_y:
#     print(le.inverse_transform([np.argmax(i)]))

# create test and train set
X_train, X_test = train_test_split(
    history_tfidf,
    test_size=0.20,
    random_state=42069
)
y_train, y_test = train_test_split(
    onehot_y,
    test_size=0.20,
    random_state=42069
)


In [73]:
X_train.shape

(903, 5831)

In [74]:
X_test.shape

(226, 5831)

In [82]:
classifier = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy')
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=1000)

In [83]:
y_pred = classifier.predict(X_test)
from sklearn import metrics 
print()
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
#Need to beat 74% Accuracy (100 trees) in Tuning 
# ACCURACY OF THE MODEL:  0.7433628318584071 with 10000


ACCURACY OF THE MODEL:  0.7522123893805309


In [77]:
df.creator.value_counts() #1129 - Naive for marvel 0.516386182462356

marvel comics        583
dc comics            414
shueisha              35
dark horse comics     26
lego                  22
george lucas          18
image comics          16
nbc - heroes          15
Name: creator, dtype: int64

In [79]:
#Tuning Parameters 
# n_estimators : the number of trees in the forest of the model
# max_depth: maximum depth of each tree, dont touch
# min_samples_split: minimum number of samples required to split an internal leaf node.
# min_samples_leaf: minimum number of samples required to be at a leaf node.
#criterion{“gini”, “entropy”}, default=”gini”

0.516386182462356

In [90]:
param_grid = { 
    'n_estimators': [200, 500, 1000],
    #'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8]
}

In [ ]:
rfc=RandomForestClassifier(random_state=545)

RF_grid = GridSearchCV(rfc, param_grid=param_grid)
RF_grid.fit(X_train, y_train)


In [92]:
RF_grid.best_params_


{'max_depth': 8, 'n_estimators': 200}

In [93]:
classifier = RandomForestClassifier(n_estimators = 200, max_depth = 8)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn import metrics 
print()
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
#Need to beat 74% Accuracy (100 trees) in Tuning 
# ACCURACY OF THE MODEL:  0.7433628318584071 with 10000


ACCURACY OF THE MODEL:  0.5752212389380531
